# Graph-RandomWalk 기반 개인화

참고. [cs246](http://web.stanford.edu/class/cs246/info.html)

## 용어
- bipartite graph: 그래프에서 성격이 다른 두개의 vertext 셋이 존재. 참고[Wikipedia](https://en.wikipedia.org/wiki/Bipartite_graph)
    - U: 사용자 히스토리에 관계된 vertex 셋
    - V: 위픽 세팅에 관계된 vertex 셋
    - U, V 의 intra-connection은 없음, inter-connection만 존재
- undirected edge between u and v: 사용자 히스토리의 딜 u와 위픽 세팅 v 간 유사도
    - f_u: 딜 u의 임베딩 벡터 (word2vec을 사용, unit-length in L2-norm)
    - f_v: 딜 v의 임베딩 벡터
    - s(u,v) = $\exp(\alpha \times f_u \cdot f_v)$ : 유사도
- P: u -> v로 전이할 확률
    - $\sum_{v} P_{uv} = 1$ : s(u,v)를 가공하여 생성 
- Q: v -> u로 전이할 확률
    - $\sum_{u} Q_{vu} = 1$ : s(u,v)를 가공하여 생성
    
## 방법
- v = (1,1, ..., 1) / |V| 로 초기화
- v' = v x Q x P 를 반복 (RANDOM WALK 수행)
    - 수렴할 때까지 (금새 수렴)
- 또는 QP 행렬의 first-left-eigenvector를 찾는다.

## 해석
- 최종 찾아진 v는 Page Rank와 유사하게, V 셋 내 deal의 중요도로 해석할 수 있다.
- 내림차순으로 정렬하여 serving!!!

In [79]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from elasticsearch.helpers import bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime
from pymongo import MongoClient
import pymongo
import pandas as pd
import numpy as np
from scipy.sparse.linalg import eigs, svds
from operator import itemgetter

In [4]:
### twiceSpark1
es_url = '10.102.50.47:9200'
es = elasticsearch.Elasticsearch(es_url)
D = 100

## ES 관련 루틴


In [5]:
def es_search_dids_for_user(user_id, day_limit, gte_slot=1, ignore_consecutives=False):
    """
    user_id의 모든 v 가져오기
    day_limit 이전 것만 가져온다.
    return
    - 1st: v의 set
    - 2nd: 확장 정보 (v, rgtime, slot)
    """
    res = es.search(index='wepick_seq',
                  body={
                    "query": {
                      "bool": {
                        "must": {
                          "term": {"u": user_id}
                        },
                        "filter": {
                          "range": {
                            "rgtime": {
                              "lt": day_limit
                            }
                          }
                        }
                      }
                    },
                    "size": 64,
                    "sort": {"rgtime": "desc"}
                  }
                  )
    if res['hits']['total'] > 0:
        until_dt = pd.to_datetime(day_limit).to_pydatetime()
        filtered = []
        prev_v = None
        for hit in res['hits']['hits']:
            if ignore_consecutives == False or (prev_v is not None and prev_v != hit['_source']['v']):
                filtered.append((hit['_source']['v'], hit['_source']['rgtime'], hit['_source']['slot']))
            prev_v = hit['_source']['v']
        return set(map(lambda x: x[0], filtered)), filtered
    return None, None

In [6]:
def es_gather_word2vec_raw(dids):
    """
    dids로부터, word2vec을 모은다.
    return
    - dids: unit-length w2v (normalized by L2-norm)
    """
    res = es.search(index='deal_word2vec', 
                body={
                    'from':0, 'size': len(dids),
                    "_source": ["values", "v"],
                    'query': {
                        'ids': {'values': dids }
                        }                        
                    }
               )
    dic = {}
    zerovec = np.zeros((D,), dtype=np.float32)
    for hit in res['hits']['hits']:
        did = hit['_source']['v']
        vec = np.array(hit['_source']['values'])
        if len(vec) > 0 and np.allclose(vec, zerovec) == False:
            vec /= np.sqrt(np.sum(vec**2))
            dic[did] = vec
    return dic


In [7]:
def es_read_wepick_setting(dt, start_slot=20):
    """
    위픽 세팅 로딩
    """
    res = es.search(index='wepick_setting_ext', 
                body={
                    'query': {
                        'term': {'dt': dt }
                        }                        
                    }
               )
    if res['hits']['total'] > 0:
        dic = {}
        vec = []
        for s in res['hits']['hits'][0]['_source']['settings']:
            if s['slot'] >= start_slot:
                dic[s['slot']] = s['did']
                vec.append(s['did'])
        return vec, dic
    return None, None

In [8]:
def es_scan_extra_by_dids(dids):
    """
    dids로부터, mn, tn1를 가져온다.
    """
    res = es.search(index='dealinfos', 
                body={
                    'from':0, 'size': len(dids),
                    "_source": ["mn", "tn1", "did"],
                    'query': {
                        'ids': {'values': dids }
                        }                        
                    }
               )
    dic = {}
    for hit in res['hits']['hits']:
        dic[hit['_source']['did']] = (hit['_source']['mn'], hit['_source']['tn1'])
    return dic

### mongoDB for ActionInfos2

In [9]:
client = MongoClient(host='35.190.239.204', port=27017, username='praha_read', password='praha!@#', authSource='praha')

db = client['praha']

col = db['memberActionInfos2']

In [10]:
def mg_get_ordered_dids(mid, lt_day="20180411", limit=32):
    """
    구매한 딜들을 조회
    """
    result = col.find({"mid":mid, 'ft.o':{"$ne":[]}, 'day':{"$lt":lt_day}}, {'day':1, 'ft.o': 1, '_id':0}).sort('day', pymongo.DESCENDING).limit(limit)
    out = set()
    for res in result:
        out.update(list(map(lambda x: x['did'], res['ft']['o'])))
    return out

In [11]:
def mg_get_clicked_dids(mid, lt_day="20180411", limit=32, use_search_induced_click=False):
    """
    클릭한 딜들을 조회
    """
    result = col.find({"mid":mid, 'ft.c':{"$ne":[]}, 'day':{"$lt":lt_day}}, {'day':1, 'ft.c': 1, '_id':0}).sort('day', pymongo.DESCENDING).limit(limit)
    out = set()
    for res in result:
        out.update(list(map(lambda x: x['did'], 
                            res['ft']['c'] if use_search_induced_click == False else filter(lambda x: x['s'] != '', res['ft']['c'])
                           )))
    return out

## Ranking 관련

In [12]:
def get_refined_scores(scores, extra_dic):
    refined_scores = []
    for did, score in scores:
        if did in extra_dic:
            refined_scores.append((score, did, extra_dic[did][0], extra_dic[did][1]))
        else:
            refined_scores.append((score, did, "", ""))
    return refined_scores

In [13]:
def print_result(out, wepick_slot_dic):
    for s, did, title, cate in out:
        org_slot = wepick_slot_dic[did] if did in wepick_slot_dic else -1
        print("{:0.4f}, {}, {}, {}, {}".format(s, did, title, org_slot, cate))
            

In [149]:
def print_user_history(ex):
    """
    유저 히스토리 출력
    """
    dic = es_scan_extra_by_dids(list(map(lambda x: x[0], ex)))
    for did, ts, slot in ex:
        if did in dic:
            print("{}, {:.16}, {:2d}, {}".format(did, ts, slot, dic[did][0]))
        else:
            print("{}, {:.16}, {:2d}".format(did, ts, slot))
#         if did in dic:
#             print("{}, {:.16}, {:2d}, {}, {}".format(did, ts, slot, dic[did][0], dic[did][1]))
#         else:
#             print("{}, {:.16}, {:2d}".format(did, ts, slot))
    

## Random Walk 관련

In [15]:
def get_w2vec_matrix(A):
    """
    A = dict: dealidx -> np.array((D,), dtype=np.float32)
    """
    
    deal_to_idx =  {k:i for i, k in enumerate(A.keys())}
    idx_to_deal =  {i:k for i, k in enumerate(A.keys())}
    
    M = np.array(list(A.values()), dtype=np.float32)
    return M, idx_to_deal

In [16]:
def get_random_walk_matrix(U, V, alpha=5.0):
    """
    V->U->V 로 전이시켜주는 확률 매트릭스를 리턴한다고 생각하자.
    """
    P_ = np.matmul(U, V.T)
    Q_ = np.matmul(V, U.T)
    
    P = np.exp(alpha*P_)
    Q = np.exp(alpha*Q_)

    P = P / np.sum(P, axis=1, keepdims=True)
    Q = Q / np.sum(Q, axis=1, keepdims=True)
    
    return np.matmul(Q, P)    

In [144]:
def do_random_walk(QQ, prior_v=None, tol=1e-8):
    """
    QQ: |V| x |V| matrix
    tol 미만 될 때까지 반복
    시작: uniform probability가 되게 설정
    """
    y = np.ones((1, QQ.shape[0]), dtype=np.float32) / QQ.shape[0] if prior_v is None else prior_v
    while True:
        new_y = np.dot(y, QQ)
        if np.linalg.norm(y - new_y) < tol:
            return new_y.squeeze()
        y = new_y   

In [136]:
def solve_random_walk_analytically(QQ):
    """
    QQ_ij = transition probability from i to j
    scipy 의 eigs를 통한 솔루션 구하기
    """
    w, v = eigs(QQ.T, k=1)
    y = v.real
    return (y / y.sum()).squeeze()

In [138]:
def calc_score_by_random_walk(W, prior_v=None, solve_analytically=False):
    y = solve_random_walk_analytically(W) if solve_analytically else do_random_walk(W, prior_v)
    sorted_ii = np.argsort(y)[::-1]
    scores = list(map(lambda x: (jj[x], y[x]), sorted_ii))
    return scores

## BiRank 알고리즘 (확장)
- 참고 [논문링크](https://arxiv.org/pdf/1708.04396.pdf)
    - 4페이지 참고
- 내용
    - [HITS 알고리즘 변형] (https://en.wikipedia.org/wiki/HITS_algorithm)
    - u, v 를 번갈아며 RandomWalk

In [20]:
def get_W_for_birank(U, V, scale=5.0):
    return np.exp(scale * np.matmul(U, V.T))    

In [21]:
def get_prior_for_birank(W):
    """
    prior 값 계산 (베이직)
    """
    n_u, n_v = W.shape[0], W.shape[1]
    u_0 = np.ones((n_u,), dtype=np.float32) / n_u
    v_0 = np.ones((n_v,), dtype=np.float32) / n_v

    return u_0, v_0

In [22]:
def iterative_birank_algorithm(W, alpha=0.95, beta=0.95, tol=1e-5):
    Du = np.sum(W, axis=1)**-(0.5)
    Dv = np.sum(W, axis=0)**-(0.5)
    S = np.matmul(np.matmul(np.diag(Du), W), np.diag(Dv))
    

    u_0, v_0 = get_prior_for_birank(W)
    
    n_u, n_v = W.shape[0], W.shape[1]
    u = np.ones((n_u,), dtype=np.float32) / n_u
    v = np.ones((n_v,), dtype=np.float32) / n_v

    tol = 1e-4
    while True:
        new_v = alpha * np.dot(S.T, u) + (1-alpha)*v_0
        new_u = beta * np.dot(S, v) + (1-beta)*u_0
        if np.linalg.norm(v - new_v) < tol and np.linalg.norm(u - new_u) < tol:
            break
        v = new_v
        u = new_u
    return u, v

In [23]:
def calc_score_by_birank_algorithm(W, prior_v=None):
    u, v = iterative_birank_algorithm(W)
    sorted_ii = np.argsort(v)[::-1]
    scores = list(map(lambda x: (jj[x], v[x]), sorted_ii))
    return scores

### Wepick Setting load

In [24]:
# 2018-04-11 21 시의 위픽 세팅 로딩
wepick_setting, wepick_dic = es_read_wepick_setting('2018-04-11 21')

wepick_slot_dic = dict(zip(wepick_dic.values(), wepick_dic.keys()))

### deal_profile loading

In [25]:
# 위픽 세팅에 따른 딜들에 대한 deal_profile을 생성
deal_profile_dic = es_gather_word2vec_raw(wepick_setting)
extra_dic = es_scan_extra_by_dids(wepick_setting)

## 3월 11 -  4월 10일까지 위픽 클릭 데이터에 대해 구성한 user_profile에 대한 테스트

In [26]:
deals_user_viewed, ex = es_search_dids_for_user(1000007, '2018-04-11', ignore_consecutives=False)

user_profile_dic = es_gather_word2vec_raw(list(deals_user_viewed))

In [150]:
print_user_history(ex)

3513125, 2018-04-10T12:35,  1, [슈퍼투데이특가] 호텔수건 170g10장
3513125, 2018-04-10T12:34,  1, [슈퍼투데이특가] 호텔수건 170g10장
3513125, 2018-04-10T12:28,  1, [슈퍼투데이특가] 호텔수건 170g10장
3513125, 2018-04-10T10:29,  1, [슈퍼투데이특가] 호텔수건 170g10장
3513125, 2018-04-10T09:16,  1, [슈퍼투데이특가] 호텔수건 170g10장
3513125, 2018-04-10T09:15,  1, [슈퍼투데이특가] 호텔수건 170g10장
3513125, 2018-04-10T09:14,  1, [슈퍼투데이특가] 호텔수건 170g10장
3523535, 2018-04-09T23:10, 29, [위메프] 10만 포인트+5,000P
3503625, 2018-04-05T05:41,  1, [무료배송] 4월의 아디다스 KIDS
3503625, 2018-04-05T05:40,  1, [무료배송] 4월의 아디다스 KIDS
3499380, 2018-04-05T05:28,  4, [플레이특가] 원마운트 워터파크 1+1
3499380, 2018-04-05T03:00,  4, [플레이특가] 원마운트 워터파크 1+1
3503625, 2018-04-05T03:00,  1, [무료배송] 4월의 아디다스 KIDS
3504535, 2018-04-05T02:53,  6, [투데이특가] 라피타제모기 런칭행사
3499380, 2018-04-05T02:47,  4, [플레이특가] 원마운트 워터파크 1+1
3499380, 2018-04-05T02:47,  4, [플레이특가] 원마운트 워터파크 1+1
3503625, 2018-04-05T01:57,  1, [무료배송] 4월의 아디다스 KIDS
3493647, 2018-04-04T12:09,  1, [투데이특가] 챔피온 반팔티셔츠
3476054, 2018-04-04T07:28,  1, [투데이특가] 롯데마트 모바일금액권

In [157]:
for x in map(lambda x: x[0], extra_dic.values()): print(x)

사이판 월드 골드 4/5일+마나가하
휴비딕 체온계 & 탕온도 특가전
[무료배송] 롱티/티셔츠/원피스
[게릴라특가] 궁중 너비아니 4팩
[20%쿠폰] 니베아 립밤 바디로션 
[투데이특가] 맑음 배도라지 50팩
[리빙위크] 비즈니스보루네오 소파
[6천원쿠폰] 기습쿠폰전 오늘마지막!
[투데이특가] 더사랑이 여름 아동복
[투데이특가] 우리밀 아이국수 8인분
[무료배송] 봄 아동복 브랜드 연합전
[투데이특가] 1+ 한우 제비추리150g
[역시트롬] LG 드럼세탁기 모음
[투데이특가] 오뚜기 컵누들 10+5개
[무료배송] 프롬유 ~20%할인쿠폰
[플레이특가] 아티제 모바일상품권
[컬러풀] GTX1060 6GB 그래픽카드
[리빙위크] 사이오스 샴푸 + 증정품
[심야특가] 파파야 여성 의류 모음전
[무료배송] 까만색 잇몸치약 130g 
[투데이특가] 꺾어먹는 비요뜨 12개
[무료배송] 입맛살리는 명란파치 1kg
[투데이특가] 닥스셔츠 긴/반팔 BEST
[게릴라특가] 판퍼즐&코코몽자석놀이
[원더쿠폰] 과탄산소다 베이킹소다
[사은품증정] 리빙웰 에어프라이어
[투데이특가] 보노바스켓 정리함 80L
[투데이특가] 니트/가디건/원피스 외
[리빙위크] 3M 정전기청소포/물걸레
[투데이특가] 샤오미 공기청정기
[할인사건] 홍콩VS마카오 항공&자유
[투데이특가] 장조림 2팩+ 본죽 2팩!
[투데이특가] 닥터마밍 물티슈 캡×10
[투데이특가] 좋은느낌 생리대 38+38
[무료배송] 빅사이즈/원피스/롱티
[투데이특가] 여심저격 클루나드시계
[가전쿠폰] LG 13kg 통돌이세탁기
[위메프] 10만 포인트+5,000P
[원더쿠폰] 봄 귀걸이,추가할인쿠폰
[투데이특가] 쁘띠첼 과일젤리 15개
[엄마니까] 쁘띠뮤 여름 횡재가격!
아임닭 닭가슴살 슬라이스 15+1
[투데이특가] 더수 몰랑 물티슈 10팩
[하객패션] 락피쉬 18년S/S+20%쿠폰!
[투데이특가] 금구운란 30구 1판
[리빙위크] 쉬즈홈 Best 이불 커튼
[롯데] 르까프 아동/성인 빅세일
[블랙데이] 짜장면 치즈 케이크 大
[투데이특가

In [28]:
U, ii = get_w2vec_matrix(user_profile_dic)
V, jj = get_w2vec_matrix(deal_profile_dic)

In [92]:
W = get_random_walk_matrix(U, V)

In [145]:
out = get_refined_scores(calc_score_by_random_walk(W), extra_dic)

In [146]:
print_result(out, wepick_slot_dic)

0.0550, 3515524, [무료배송] 롱티/티셔츠/원피스, 45, 티셔츠
0.0442, 3512593, [무료배송] 봄 아동복 브랜드 연합전, 36, 아동공용의류
0.0435, 3527477, [투데이특가] 니트/가디건/원피스 외, 61, 원피스
0.0417, 3527575, [무료배송] 프롬유 ~20%할인쿠폰, 55, 티셔츠
0.0385, 3525317, [무료배송] 빅사이즈/원피스/롱티, 28, 원피스
0.0383, 3525500, [하객패션] 포커스 봄구성완벽해, 74, 티셔츠
0.0383, 3541064, [위메프] 10만 포인트+5,000P, 33, 온라인 이용권
0.0381, 3522402, [무료배송] 에비수 본사특가 20%쿠폰, 44, 티셔츠
0.0331, 3514459, [심야특가] 파파야 여성 의류 모음전, 32, 티셔츠
0.0273, 3512215, [6천원쿠폰] 기습쿠폰전 오늘마지막!, 30, 색조메이크업
0.0254, 3513766, [어린이날] 해피버스 7부/자가드내의, 75, 내의/잠옷/속옷
0.0248, 3525068, [엄마니까] 쁘띠뮤 여름 횡재가격!, 66, 아동공용의류
0.0239, 3492158, [할인사건] 홍콩VS마카오 항공&자유, 77, 홍콩
0.0233, 3527053, [투데이특가] 더사랑이 여름 아동복, 43, 아동공용의류
0.0196, 3486081, 사이판 월드 골드 4/5일+마나가하, 22, 사이판
0.0135, 1438471, [20%쿠폰] 니베아 립밤 바디로션 , 34, 립케어
0.0134, 3528363, [하객패션] 백화점 잡화 267종! +20%, 27, 벨트
0.0133, 3515873, [투데이특가] 레이스 덧신 1+1+1, 65, 스타킹/양말
0.0124, 3515690, [투데이특가] 닥스셔츠 긴/반팔 BEST, 81, 셔츠/남방
0.0124, 3524547, [투데이특가] 1+ 한우 제비추리150g, 58, 한우
0.0123, 3522395, [롯데] 르까프 아동/성인 빅세일, 46,

### birank 알고리즘 결과 (참고)

In [29]:
out = get_refined_scores(calc_score_by_birank_algorithm(get_W_for_birank(U, V)), extra_dic)

In [30]:
print_result(out, wepick_slot_dic)

0.0326, 3515524, [무료배송] 롱티/티셔츠/원피스, 45, 티셔츠
0.0294, 3512593, [무료배송] 봄 아동복 브랜드 연합전, 36, 아동공용의류
0.0290, 3527477, [투데이특가] 니트/가디건/원피스 외, 61, 원피스
0.0284, 3527575, [무료배송] 프롬유 ~20%할인쿠폰, 55, 티셔츠
0.0282, 3541064, [위메프] 10만 포인트+5,000P, 33, 온라인 이용권
0.0274, 3525317, [무료배송] 빅사이즈/원피스/롱티, 28, 원피스
0.0274, 3522402, [무료배송] 에비수 본사특가 20%쿠폰, 44, 티셔츠
0.0274, 3525500, [하객패션] 포커스 봄구성완벽해, 74, 티셔츠
0.0255, 3514459, [심야특가] 파파야 여성 의류 모음전, 32, 티셔츠
0.0240, 3512215, [6천원쿠폰] 기습쿠폰전 오늘마지막!, 30, 색조메이크업
0.0230, 3513766, [어린이날] 해피버스 7부/자가드내의, 75, 내의/잠옷/속옷
0.0227, 3525068, [엄마니까] 쁘띠뮤 여름 횡재가격!, 66, 아동공용의류
0.0227, 3492158, [할인사건] 홍콩VS마카오 항공&자유, 77, 홍콩
0.0219, 3527053, [투데이특가] 더사랑이 여름 아동복, 43, 아동공용의류
0.0206, 3486081, 사이판 월드 골드 4/5일+마나가하, 22, 사이판
0.0172, 1438471, [20%쿠폰] 니베아 립밤 바디로션 , 34, 립케어
0.0169, 3515873, [투데이특가] 레이스 덧신 1+1+1, 65, 스타킹/양말
0.0169, 3528363, [하객패션] 백화점 잡화 267종! +20%, 27, 벨트
0.0166, 3524547, [투데이특가] 1+ 한우 제비추리150g, 58, 한우
0.0164, 3522395, [롯데] 르까프 아동/성인 빅세일, 46, 남성 티셔츠/상의 기타
0.0163, 3527569, [투데이특가] 아디다스 그래픽스케일,

## 구매 did로 부터 랭킹 테스트

- 구매 did 들의 word2vec을 사용

In [31]:
deals_user_purchased = mg_get_ordered_dids(1000007, limit=32)
user_profile_dic = es_gather_word2vec_raw(list(deals_user_purchased))

In [32]:
U, ii = get_w2vec_matrix(user_profile_dic)
V, jj = get_w2vec_matrix(deal_profile_dic)

In [33]:
W = get_random_walk_matrix(U, V)

In [34]:
out = get_refined_scores(calc_score_by_random_walk(W), extra_dic)

In [35]:
print_result(out, wepick_slot_dic)

0.0407, 3513766, [어린이날] 해피버스 7부/자가드내의, 75, 내의/잠옷/속옷
0.0342, 3525068, [엄마니까] 쁘띠뮤 여름 횡재가격!, 66, 아동공용의류
0.0281, 3529131, 휴비딕 체온계 & 탕온도 특가전, 24, 유아건강/유아위생용품
0.0278, 3527053, [투데이특가] 더사랑이 여름 아동복, 43, 아동공용의류
0.0259, 3500355, [투데이특가] 좋은느낌 생리대 38+38, 47, 화이트
0.0245, 3532552, [투데이특가] 맑음 배도라지 50팩, 86, 아기간식/아기음료
0.0185, 3464309, [쿠폰할인] LG 공기청정기 AS181DAW, 57, 공기청정기
0.0184, 3522395, [롯데] 르까프 아동/성인 빅세일, 46, 남성 티셔츠/상의 기타
0.0178, 3519047, [투데이특가] 풀무원 간편국15+5입, 31, 즉석밥/국/카레
0.0169, 3527569, [투데이특가] 아디다스 그래픽스케일, 91, 반팔 티셔츠
0.0169, 3539755, [게릴라특가] 판퍼즐&코코몽자석놀이, 41, 유아퍼즐
0.0163, 3515873, [투데이특가] 레이스 덧신 1+1+1, 65, 스타킹/양말
0.0163, 3544419, [게릴라특가] 봄맞이 카페트 150x200, 71, 카페트/러그
0.0161, 3518630, [투데이특가] 꺾어먹는 비요뜨 12개, 79, 두유/우유
0.0157, 1438471, [20%쿠폰] 니베아 립밤 바디로션 , 34, 립케어
0.0156, 3512215, [6천원쿠폰] 기습쿠폰전 오늘마지막!, 30, 색조메이크업
0.0152, 3504137, [리빙위크] 비즈니스보루네오 소파, 78, 소파
0.0151, 3507588, [투데이특가] 보노바스켓 정리함 80L, 87, 리빙박스/수납함
0.0150, 3538332, [리빙위크] 사이오스 샴푸 + 증정품, 26, 샴푸/린스
0.0148, 3526985, [원더쿠폰] 봄 귀걸이,추가할인쿠폰, 42, 귀걸이
0

## 클릭 did로 부터 랭킹 테스트

- 유저가 클릭한 did 들의 word2vec을 사용
- use_search_induced_click==True 면, 검색후 클릭된 did만 가져온다.

In [36]:
deals_user_search_click= mg_get_clicked_dids(1000007, limit=32, use_search_induced_click=True)
user_profile_dic = es_gather_word2vec_raw(list(deals_user_search_click))

In [37]:
U, ii = get_w2vec_matrix(user_profile_dic)
V, jj = get_w2vec_matrix(deal_profile_dic)

In [38]:
W = get_random_walk_matrix(U, V)

In [39]:
out = get_refined_scores(calc_score_by_random_walk(W), extra_dic)

In [40]:
print_result(out, wepick_slot_dic)

0.0379, 3541064, [위메프] 10만 포인트+5,000P, 33, 온라인 이용권
0.0362, 3521785, [쿠폰할인] 중고폰 노트5/S7/엣지, 51, 공기계-미사용/미개봉
0.0269, 3504137, [리빙위크] 비즈니스보루네오 소파, 78, 소파
0.0260, 3512215, [6천원쿠폰] 기습쿠폰전 오늘마지막!, 30, 색조메이크업
0.0254, 3464309, [쿠폰할인] LG 공기청정기 AS181DAW, 57, 공기청정기
0.0230, 3544419, [게릴라특가] 봄맞이 카페트 150x200, 71, 카페트/러그
0.0218, 3527569, [투데이특가] 아디다스 그래픽스케일, 91, 반팔 티셔츠
0.0209, 3511172, [컬러풀] GTX1060 6GB 그래픽카드, 80, 그래픽카드
0.0207, 3061867, [추가쿠폰] 삼성 냉장고 RH81K8050SA, 50, 양문형 냉장고
0.0201, 3507588, [투데이특가] 보노바스켓 정리함 80L, 87, 리빙박스/수납함
0.0199, 3509649, [리빙위크] 리빙숲 리빙박스72L 4개, 49, 리빙박스/수납함
0.0191, 1438471, [20%쿠폰] 니베아 립밤 바디로션 , 34, 립케어
0.0181, 3512421, [투데이특가] 휴대용 핸디 선풍기!, 85, 휴대용 선풍기
0.0180, 3521723, [투데이특가] 샤오미 공기청정기, 25, 공기청정기
0.0176, 2258611, [사은품증정] 리빙웰 에어프라이어, 84, 튀김기
0.0170, 3068897, [가전쿠폰] LG 13kg 통돌이세탁기, 37, 일반 세탁기
0.0169, 3119252, [역시트롬] LG 드럼세탁기 모음, 60, 드럼세탁기
0.0169, 3527675, [투데이특가] 파크론 주방매트 대+소, 20, 카페트/러그
0.0161, 3544562, [게릴라특가] 실리콘 주방용품 모음, 40, 조리도구
0.0161, 3524547, [투데이특가] 1+ 한우 제비추리150g, 58, 한우
